In [1]:
import requests
from lxml import etree
from io import StringIO
import jieba
import pandas as pd
import time
from jieba import posseg as pseg
import random

In [2]:
#指定辭典檔
jieba.set_dictionary('./mydict/dict.txt.big')
#加入自定義辭典
jieba.load_userdict('./mydict/mydict.txt')

Building prefix dict from C:\Users\leavi\Desktop\pyETL\mydict\dict.txt.big ...
Loading model from cache C:\Users\leavi\AppData\Local\Temp\jieba.uf6b4e716bf0981ff959945b45b3f2755.cache
Loading model cost 0.926 seconds.
Prefix dict has been built successfully.


In [3]:
# 開檔
fileAllLines = []
#避免檔案過大造成記憶體不足使用 with open 及 read()控制讀取數量
with open('./file/healthy_all.txt','r',encoding="utf-8") as fileLine:
    for line in iter(lambda: fileLine.read(1024), ''):
        #print(line)
        fileAllLines.append(line)
        
new_line = ' '.join(fileAllLines)
#new_line

In [34]:
#加入stop word
with open(file='./mydict/stop_words.txt', mode='r', encoding='utf-8') as file:
    stop_words = file.read().split('\n')
    
stop_words.append('\n')  ## 文章中有許多分行符號，這邊加入停用字中，可以把它拿掉
stop_words.append('\n\n')
stop_words


['the',
 'of',
 'is',
 'and',
 'to',
 'in',
 'that',
 'we',
 'for',
 'an',
 'are',
 'by',
 'be',
 'as',
 'on',
 'with',
 'can',
 'if',
 'from',
 'which',
 'you',
 'it',
 'this',
 'then',
 'at',
 'have',
 'all',
 'not',
 'one',
 'has',
 'or',
 'that',
 '\ufeff,',
 '?',
 '、',
 '。',
 '“',
 '”',
 '《',
 '》',
 '！',
 '，',
 '：',
 '；',
 '？',
 '人民',
 '末##末',
 '啊',
 '阿',
 '哎',
 '哎呀',
 '哎喲',
 '唉',
 '按',
 '按照',
 '依照',
 '吧',
 '吧噠',
 '把',
 '罷了',
 '被',
 '本',
 '本著',
 '比',
 '比方',
 '比如',
 '鄙人',
 '彼',
 '彼此',
 '邊',
 '別',
 '別的',
 '別說',
 '並',
 '並且',
 '不比',
 '不成',
 '不單',
 '不但',
 '不獨',
 '不管',
 '不光',
 '不過',
 '不僅',
 '不拘',
 '不論',
 '不怕',
 '不然',
 '不如',
 '不特',
 '不惟',
 '不問',
 '不只',
 '朝',
 '朝著',
 '趁',
 '趁著',
 '乘',
 '沖',
 '除',
 '除此之外',
 '除非',
 '除了',
 '此',
 '此間',
 '此外',
 '從',
 '從而',
 '打',
 '待',
 '但',
 '但是',
 '當',
 '當著',
 '到',
 '得',
 '的',
 '的話',
 '等',
 '等等',
 '地',
 '第',
 '叮咚',
 '對',
 '對於',
 '多',
 '多少',
 '而',
 '而況',
 '而且',
 '而是',
 '而外',
 '而言',
 '而已',
 '爾後',
 '反過來',
 '反過來說',
 '反之',
 '非但',
 '非徒',
 '否則',
 '嘎',
 '嘎登',
 '該',
 

In [35]:
# 斷詞
start_time = time.time()
import jieba.posseg as pseg
#jieba.enable_paddle()
#jieba.enable_parallel(4)
seg = []
for i in range(len(fileAllLines)):
    #cut_list = list( jieba.cut(fileAllLines[i], cut_all = False) )
    #seg.append([' '.join([ item for item in cut_list if len(item) > 1 ])])
    cut_result = list(pseg.cut(fileAllLines[i], use_paddle=False))
    each_line_list = []
    for w, p in cut_result:#標註詞性
        #print("%s: %s"%(w, p))
        #if w not in stop_words and p in ['n', 'nr','ns','nt','nz','nl','ng', 't', 's','vn','v','vd','a','ad','an','d','r']:
        #上面這行加入了以詞性作為過濾的條件
        if w not in stop_words :
                #print("%s: %s"%(w, p))
            each_line_list.append(w)
        new_line = ' '.join(each_line_list)
    #print(new_line)
    seg.append(new_line)
print("--- spend %s seconds ---" % (time.time() - start_time))

print(len(seg))

--- spend 3242.4111042022705 seconds ---
10436


In [36]:
# 斷詞結果存檔
segSaveFile = 'segmentNormalDone.txt'
with open(segSaveFile, 'wb') as saveFile:
    for i in range(len(seg)):
        #words = seg[i][0].encode('utf-8')
        words = seg[i].encode('utf-8')
        if len(words) > 0:
            saveFile.write(words)
            saveFile.write('\n'.encode())

In [37]:
#載入word2vec 函式庫
from gensim.models import word2vec

In [38]:
#一行一行的從檔案中取出句子：一行一個句子，句子中的字必須預先處理好並以空白隔開
sentences = word2vec.LineSentence("segmentNormalDone.txt")

In [39]:
#訓練模型
#size: 向量維度 = 300
#sg: 0(CBOW), 1(Skip-gram)
start_time = time.time()
model = word2vec.Word2Vec(sentences, size=300, sg=1, window=10, workers=3, min_count=2)
print("--- spend %s seconds ---" % (time.time() - start_time))

model

--- spend 207.18972373008728 seconds ---


In [40]:
#儲存訓練好的word2vec 模型
model.save("word2vecNormal.model")

model.corpus_total_words


2598088

In [44]:
#測試詞相似度
model.similar_by_word('每天')

<ipython-input-44-0d78ea5d1082>:2: DeprecationWarning: Call to deprecated `similar_by_word` (Method will be removed in 4.0.0, use self.wv.similar_by_word() instead).
  model.similar_by_word('每天')


[('天天', 0.6080406904220581),
 ('混日子', 0.592911422252655),
 ('度日', 0.5820943713188171),
 ('阿門', 0.579181969165802),
 ('每天晚上', 0.5781623125076294),
 ('太爽了', 0.5763921737670898),
 ('無所事事', 0.5744073987007141),
 ('緊迫', 0.5742000937461853),
 ('抽時間', 0.5735272169113159),
 ('一兩週', 0.5726936459541321)]

In [45]:
#詞相似度
model.similar_by_word('一直')

<ipython-input-45-dc3cafe384ae>:2: DeprecationWarning: Call to deprecated `similar_by_word` (Method will be removed in 4.0.0, use self.wv.similar_by_word() instead).
  model.similar_by_word('一直')


[('繞圈圈', 0.5793926119804382),
 ('無時無刻', 0.5786219239234924),
 ('耍賴', 0.5743733644485474),
 ('討厭鬼', 0.5736790299415588),
 ('逼出來', 0.5722613334655762),
 ('使喚', 0.5674321055412292),
 ('前兆', 0.5667661428451538),
 ('氣不氣', 0.5665849447250366),
 ('痛處', 0.565506100654602),
 ('搞不清', 0.5642781257629395)]

In [46]:
#詞相似度
model.similar_by_word('總是')

<ipython-input-46-79b1978f9230>:2: DeprecationWarning: Call to deprecated `similar_by_word` (Method will be removed in 4.0.0, use self.wv.similar_by_word() instead).
  model.similar_by_word('總是')


[('認真思考', 0.6209027767181396),
 ('捉摸不定', 0.6182634830474854),
 ('自尋煩惱', 0.6158167123794556),
 ('磨光', 0.6137054562568665),
 ('太多太', 0.6103761792182922),
 ('一笑置之', 0.6096199154853821),
 ('簇擁', 0.6092824935913086),
 ('種下', 0.6088994145393372),
 ('混帳', 0.6081991195678711),
 ('問句', 0.6081584692001343)]

In [47]:
#詞相似度
model.similar_by_word('我')

<ipython-input-47-676cafd5312a>:2: DeprecationWarning: Call to deprecated `similar_by_word` (Method will be removed in 4.0.0, use self.wv.similar_by_word() instead).
  model.similar_by_word('我')


[('我自', 0.7547152042388916),
 ('送光', 0.753966212272644),
 ('我他', 0.7520837187767029),
 ('避諱', 0.7472635507583618),
 ('你們好', 0.7419684529304504),
 ('床伴', 0.7341238856315613),
 ('被寵', 0.7337935566902161),
 ('仇人', 0.7329224348068237),
 ('愧於', 0.7321110963821411),
 ('福份', 0.7313300371170044)]

In [48]:
#詞相似度
model.similar_by_word('我們')

<ipython-input-48-826da677650c>:2: DeprecationWarning: Call to deprecated `similar_by_word` (Method will be removed in 4.0.0, use self.wv.similar_by_word() instead).
  model.similar_by_word('我們')


[('倆', 0.5938891172409058),
 ('我倆', 0.5586367845535278),
 ('沒能', 0.5559946298599243),
 ('就此結束', 0.5495032072067261),
 ('談天說地', 0.5388210415840149),
 ('冥冥之中', 0.5384582281112671),
 ('分道揚鑣', 0.5373210310935974),
 ('踫', 0.5361054539680481),
 ('談論到', 0.5361049771308899),
 ('拉進', 0.5357521772384644)]

In [49]:
#詞相似度
model.similar_by_word('吃藥')

<ipython-input-49-af7367194441>:2: DeprecationWarning: Call to deprecated `similar_by_word` (Method will be removed in 4.0.0, use self.wv.similar_by_word() instead).
  model.similar_by_word('吃藥')


[('停藥', 0.8395404815673828),
 ('止痛藥', 0.8304799795150757),
 ('止痛針', 0.8045859932899475),
 ('拿藥', 0.8018680214881897),
 ('擦藥', 0.799972653388977),
 ('腎', 0.7976259589195251),
 ('中耳炎', 0.7966483235359192),
 ('開藥', 0.7949013710021973),
 ('類固醇', 0.7904659509658813),
 ('安眠藥', 0.7859857082366943)]

In [50]:
#詞相似度
model.similar_by_word('憂鬱')

<ipython-input-50-59b9c94466b4>:2: DeprecationWarning: Call to deprecated `similar_by_word` (Method will be removed in 4.0.0, use self.wv.similar_by_word() instead).
  model.similar_by_word('憂鬱')


[('易怒', 0.8091617822647095),
 ('鬱悶', 0.798853874206543),
 ('排山倒海', 0.789068877696991),
 ('焦慮症', 0.7777386903762817),
 ('頭疼', 0.7764595150947571),
 ('沈重', 0.7755311131477356),
 ('大起大落', 0.767436683177948),
 ('週期性', 0.76662278175354),
 ('心悸', 0.765610933303833),
 ('孱弱', 0.764356255531311)]

In [51]:
#詞相似度
model.similar_by_word('真的')

<ipython-input-51-2df61b77d910>:2: DeprecationWarning: Call to deprecated `similar_by_word` (Method will be removed in 4.0.0, use self.wv.similar_by_word() instead).
  model.similar_by_word('真的')


[('夭壽', 0.6880010366439819),
 ('閉關', 0.6873886585235596),
 ('有件事', 0.6823240518569946),
 ('XDDDDDDD', 0.6814631223678589),
 ('不按牌理', 0.680924654006958),
 ('還累', 0.6789827346801758),
 ('小玉', 0.6783701181411743),
 ('對號入座', 0.6782324314117432),
 ('耗死', 0.6778415441513062),
 ('真他媽', 0.6772968769073486)]